In [8]:
import pickle

import pandas as pd

PATH_DF_CACHED = '../.cache/df.pickle'
PATH_DF = '../dist/all.csv'
PATH_SW = './stopwords-id.csv' # Stop Words ISO

# with open(PATH_DF_CACHED, 'rb') as f:
#     df = pickle.load(f)

df = pd.read_csv(PATH_DF)
sw = pd.read_csv(PATH_SW)


In [9]:
from mpstemmer import MPStemmer
import string

stemmer = MPStemmer()
stemmer.check_nonstandard = False
stem = stemmer.stem_kalimat

translation = str.maketrans('', '', string.punctuation)
sw_list = list(sw['stopword'])
def preprocess(s):
    clean = s.translate(translation)
    stemmed = stem(clean)
    stripped = stemmed.strip()
    tokenized = stripped.split(' ')
    sw_removed = [w for w in tokenized if w not in sw_list]
    return sw_removed

df['preprocessed'] = df['headline'].apply(preprocess)

In [10]:
from gensim.models.phrases import Phrases, Phraser

phrases2 = Phrases(df['preprocessed'], min_count=1, threshold=10)
phrases3 = Phrases(phrases2[df['preprocessed']], threshold=100)

bigram = Phraser(phrases2)
trigram = Phraser(phrases3)

df['preprocessed'] = list(trigram[df['preprocessed']])

In [11]:
with open(PATH_DF_CACHED, 'wb') as f:
    pickle.dump(df, f)

In [18]:
import random

for i in range(20):
    n = random.randint(0, len(df))
    print(n)
    print(df['date'][n])
    print(df['headline'][n])
    print(df['preprocessed'][n])

272077
2022-06-04
Ratu Elizabeth II Akhirnya Bertemu Lilibet, Putri Harry-Meghan
['ratu_elizabeth', 'ii', 'bertemu', 'lilibet', 'putri', 'harrymeghan']
847839
2022-12-29
Akademisi Dorong Pemanfaatan Produk Tembakau Alternatif untuk Kurangi Prevalensi Merokok
['akademisi', 'dorong', 'manfaat', 'produk_tembakau', 'alternatif', 'prevalensi', 'rokok']
841831
2022-12-23
Rekaman CCTV Brigadir J Masih Hidup Buyarkan Skenario Tembak-Menembak Ferdy Sambo
['rekam_cctv', 'brigadir_j', 'hidup', 'buyar', 'skenario_tembakmenembak', 'ferdy_sambo']
20884
2022-02-25
Dalam Sehari, Damkar Depok Evakuasi 3 Sanca dari Permukiman Warga
['sehari', 'damkar', 'depok', 'evakuasi', '3', 'sanca', 'mukim', 'warga']
1102012
2022-08-14
Pemukiman Strategis Peski Jatuh ke Tangan Rusia, Pilar Utama Ukraina di Donbas Makin Berkurang
['mukim', 'strategis', 'peski', 'jatuh', 'tangan', 'rusia', 'pilar_utama', 'ukraina', 'donbas']
576257
2022-07-26
Pemerintah Kelola Khusus Hutan Pulau Jawa, Momentum Reboisasi Lahan Kritis
[